In [ ]:
!pip install catboost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from catboost import CatBoostRanker, Pool
from sklearn.metrics import ndcg_score

In [ ]:
train_df = pd.read_csv('train_df.csv')

In [ ]:
train_df

In [ ]:
test_df = pd.read_csv('test_df.csv')

In [ ]:
test_df

# Обучение без обработки данных

**Используем catboost**

In [ ]:
groups = train_df['search_id'].unique(); len(groups)

In [ ]:
X_train = train_df.drop(columns=['search_id', 'target']).values
y_train = train_df['target'].values
groups_train = train_df['search_id'].values

X_test = test_df.drop(columns=['search_id', 'target']).values
y_test = test_df['target'].values
groups_test = test_df['search_id'].values

In [ ]:
parameters = {
    'iterations': 120,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=5'],
    'verbose': False,
    'random_seed': 0,
    'early_stopping_rounds' : 100,
    'loss_function': 'YetiRankPairwise',
    'depth': 3,
    'learning_rate': 0.1
}

In [ ]:
model = CatBoostRanker(**parameters)
model = model.fit(X_train, y_train, group_id=groups_train)

In [ ]:
ndcg_train = model.score(X_train, y_train, group_id=groups_train)
print(f"Train NDCG: {ndcg_train}")

ndcg_test = model.score(X_test, y_test, group_id=groups_test)
print(f"Test NDCG: {ndcg_test}")

# Обучение с обработкой данных

In [ ]:
to_drop = train_df.nunique().index[train_df.nunique() == 1]
train_df = train_df.drop(columns = to_drop, axis =1)
test_df = test_df.drop(columns = to_drop, axis =1)

In [ ]:
to_drop

In [ ]:
def remove_highly_correlated_features(df, df_test, threshold):
    corr_matrix = df.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
    df_filtered = df.drop(to_drop, axis=1)
    df_filtered_test = df_test.drop(to_drop, axis=1)
    print(to_drop)
    return df_filtered, df_filtered_test

threshold = 0.9

train_df, test_df = remove_highly_correlated_features(train_df, test_df, threshold)

print("Количество признаков после удаления коррелирующих:", test_df.shape[1])

In [ ]:
X_train = train_df.drop(columns=['search_id', 'target']).values
y_train = train_df['target'].values
groups_train = train_df['search_id'].values

X_test = test_df.drop(columns=['search_id', 'target']).values
y_test = test_df['target'].values
groups_test = test_df['search_id'].values

In [ ]:
model = CatBoostRanker(**parameters)
model = model.fit(X_train, y_train, group_id=groups_train)

In [ ]:
ndcg_train = model.score(X_train, y_train, group_id=groups_train)
print(f"Train NDCG: {ndcg_train}")

ndcg_test = model.score(X_test, y_test, group_id=groups_test)
print(f"Test NDCG: {ndcg_test}")

**Скор ухудшился на тех же параметрах**